# **Manual Accuracy Evaluation**

Words are manually rated on a Likert scale (1–5):

1 = Strongly Agree (Accurate)

2 = Agree (Mostly Accurate)

3 = Neutral (Unclear/Partial)

4 = Disagree (Somewhat Inaccurate)

5 = Strongly Disagree (Inaccurate)

In [ ]:
import pandas as pd

#data = pd.read_excel("Chialuka-Igbo-LLM.xlsx")
# data.to_csv("Chialuka-Igbo-LLM.xlsx.csv", index=False)

#print(data.head())

# Read the CSV back to display its head
ling_data = pd.read_excel("Chialuka-Igbo-LLM.xlsx")
ling_data.head()

# Export a smaller sample for human evaluation
sample_data = ling_data.sample(n=100, random_state=67)
sample_data.to_excel("sample_data_for_human_evaluation_100_67.xlsx", index=False)


Generate translation for the Igbo words in the sample file

In [ ]:
import openai

# Optional: set your API key here or via environment variable OPENAI_API_KEY
openai.api_key = ""

def translate_igbo_to_english(text):
    """
    Use GPT-4o-mini to translate Igbo to English
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful translator that translates Igbo to English."},
                {"role": "user", "content": f"Translate this Igbo sentence to English:\n{text}. Output only the result with no additional information"}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# Example usage
if __name__ == "__main__":
    igbo_input = "efere"
    english_output = translate_igbo_to_english(igbo_input)
    print(f"Igbo: {igbo_input}")
    print(f"English: {english_output}")


In [ ]:
import openai
import pandas as pd
from tqdm.auto import tqdm

# Optional: set your API key here or via environment variable
openai.api_key = "sk-*****"

def translate_igbo_to_english(text):
    """
    Use GPT-4o-mini to translate Igbo to English
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful translator that translates Igbo to English."},
                {"role": "user", "content": f"Translate this Igbo sentence to English:\n{text}. Output only the result with no additional information"}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

if __name__ == "__main__":
    # Example DataFrame; replace this with your actual sample_data
    sample_data = pd.read_excel("sample_data_for_human_evaluation.xlsx")

    # Initialize progress bar for pandas
    tqdm.pandas()

    # Apply the translation function to each 'word' in the DataFrame
    sample_data['translation'] = sample_data['word'].progress_apply(translate_igbo_to_english)

    # Display or save the translated DataFrame
    print(sample_data)
    sample_data.to_excel('translated_data_gpt_4o.xlsx', index=False)
    # sample_data.to_csv('translated_data_gpt_4o.csv', index=False)


  0%|          | 0/100 [00:00<?, ?it/s]

           word           wordClass  \
0   ọ̀gbụ̀gbọ̀                Noun   
1      ǹdègà              Number   
2   eboōbībī                Noun   
3     mmàaghā                Noun   
4       bawanye         Active verb   
..          ...                 ...   
95        àlà                Noun   
96    yì akwà         Active verb   
97          -go  Extensional suffix   
98  mùchi anya         Active verb   
99         ogè                Noun   

                                          definitions  \
0                                               vomit   
1                                            millions   
2               habitation; dwelling place; residence   
3              sword [weapon with a long metal blade]   
4                            increase; grow in number   
..                                                ...   
95  land; ground; floor; the part of the earth's s...   
96                                       wear a cloth   
97      perfect te

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 sample_data.to_excel('translated_data_gpt_4o.xlsx', index=False)

## **Semantic Similarity via Word Embeddings**

In [ ]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer, util

# Load a multilingual model that supports Igbo
model = SentenceTransformer('distiluse-base-multilingual-cased')

#translate



# Compute embeddings
df_sample['word_embedding'] = df_sample['word'].apply(lambda x: model.encode(x, convert_to_tensor=True))
df_sample['definition_embedding'] = df_sample['definitions'].apply(lambda x: model.encode(x, convert_to_tensor=True))

# Compute cosine similarity
df_sample['cosine_similarity'] = df_sample.apply(
    lambda row: util.cos_sim(row['word_embedding'], row['definition_embedding']).item(),
    axis=1
)

# View results
df_sample[['word', 'definitions', 'cosine_similarity']].head()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 910.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/rust_model.ot:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

,word,definitions,cosine_similarity
0,ọ̀gbụ̀gbọ̀,vomit,0.425870
1,ǹdègà,millions,0.357858
2,eboōbībī,habitation; dwelling place; residence,0.096069
3,mmàaghā,sword [weapon with a long metal blade],0.130959
4,bawanye,increase; grow in number,0.122570


Here is an explanation of each line in the code cell `EdA3IYKdKG89`:

*   `!pip install -q sentence-transformers`: This line uses a Colab magic command (`!`) to run a shell command. It installs the `sentence-transformers` library, which is used for generating sentence embeddings. The `-q` flag makes the installation quiet, meaning it won't print as much output.
*   `from sentence_transformers import SentenceTransformer, util`: This line imports the necessary classes and functions from the `sentence-transformers` library: `SentenceTransformer` for loading the pre-trained model and `util` for utility functions like cosine similarity calculation.
*   `model = SentenceTransformer('distiluse-base-multilingual-cased')`: This line loads a pre-trained sentence transformer model. The model `'distiluse-base-multilingual-cased'` is a multilingual model that can handle text in many different languages, including Igbo.
*   `df_sample = data_csv[['word', 'definitions']].dropna().sample(100, random_state=42).reset_index(drop=True)`: This line prepares a sample of your data for analysis.
    *   `data_csv[['word', 'definitions']]`: Selects the 'word' and 'definitions' columns from the `data_csv` DataFrame.
    *   `.dropna()`: Removes any rows that have missing values in either of these columns.
    *   `.sample(100, random_state=42)`: Randomly selects 100 rows from the remaining data. `random_state=42` ensures that you get the same sample every time you run the code.
    *   `.reset_index(drop=True)`: Resets the DataFrame index after sampling and drops the old index.
*   `df_sample['word_embedding'] = df_sample['word'].apply(lambda x: model.encode(x, convert_to_tensor=True))`: This line computes the sentence embedding for each 'word' in the `df_sample` DataFrame.
    *   `.apply(lambda x: ...)`: Applies a function to each element in the 'word' column.
    *   `model.encode(x, convert_to_tensor=True)`: Uses the loaded `SentenceTransformer` model to encode the word `x` into a numerical vector (embedding). `convert_to_tensor=True` ensures the output is a PyTorch tensor.
    *   `df_sample['word_embedding'] = ...`: Stores the resulting embeddings in a new column called 'word_embedding'.
*   `df_sample['definition_embedding'] = df_sample['definitions'].apply(lambda x: model.encode(x, convert_to_tensor=True))`: This line does the same as the previous line but for the 'definitions' column, creating a new column called 'definition_embedding' with the embeddings of the definitions.
*   `df_sample['cosine_similarity'] = df_sample.apply(lambda row: util.cos_sim(row['word_embedding'], row['definition_embedding']).item(), axis=1)`: This line calculates the cosine similarity between the word embedding and the definition embedding for each row.
    *   `.apply(lambda row: ..., axis=1)`: Applies a function to each row of the DataFrame.
    *   `util.cos_sim(row['word_embedding'], row['definition_embedding'])`: Calculates the cosine similarity between the word embedding and the definition embedding for the current row. Cosine similarity is a measure of how similar two non-zero vectors are.
    *   `.item()`: Extracts the scalar value from the resulting tensor.
    *   `df_sample['cosine_similarity'] = ...`: Stores the calculated cosine similarity in a new column called 'cosine_similarity'.
*   `df_sample[['word', 'definitions', 'cosine_similarity']].head()`: This line displays the first few rows of the `df_sample` DataFrame, showing the original word and definition along with their calculated cosine similarity.

In [ ]:
import openai

# Optional: set your API key here or via environment variable OPENAI_API_KEY
openai.api_key = "sk-******"

def translate_igbo_to_english(text):
    """
    Use GPT-4o-mini to translate Igbo to English
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful translator that translates Igbo to English."},
                {"role": "user", "content": f"Translate this Igbo sentence to English:\n{text}. Output only the result with no additional information"}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# Example usage
if __name__ == "__main__":
    igbo_input = "efere"
    english_output = translate_igbo_to_english(igbo_input)
    print(f"Igbo: {igbo_input}")
    print(f"English: {english_output}")


Machine eval of LLM translations

In [ ]:
import openai
import pandas as pd
from tqdm.auto import tqdm

# Optional: set your API key here or via environment variable
openai.api_key = "sk-****"

In [ ]:
sample_data = pd.read_excel("translated_data_gpt_4o_eval.xlsx")

In [ ]:
sample_data

,word,wordClass,definitions,pronunciation,dialects,examples,variations,tenses.infinitive,tenses.imperative,tenses.simplePast,tenses.presentPassive,tenses.simplePresent,tenses.presentContinuous,tenses.future,translation,Human Scoring,Unnamed: 16
0,ọ̀gbụ̀gbọ̀,Noun,vomit,5f90c35e49f7e863e92b7837.webm,"[{'dialects': ['Nsa'], 'pronunciation': None, ...",[{'igbo': 'Ọgbụgbọ ya na-esi mmanya mmanya....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,smoking.,0,NaN
1,ǹdègà,Number,millions,635315df8bdfdc14e3adf505.mp3,"[{'dialects': ['Owere'], 'pronunciation': '635...",[{'igbo': 'Ọchịcha ndị ahu bụrụ ụmụ na ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Greetings.,0,NaN
2,eboōbībī,Noun,habitation; dwelling place; residence,5f90c35e49f7e863e92b722c.webm,[],[{'igbo': 'Gọọmentị na-arụrụ ndịọrụ ya...,NaN,iebe ōbibi,ebee ōbibi,ebere ōbibi,NaN,na-eebe ōbibi,na-eebe ōbibi,ga-eebe ōbibi,destruction,0,NaN
3,mmàaghā,Noun,sword [weapon with a long metal blade],63593325625b93a5ab2a43da.mp3,"[{'dialects': ['Ọkụzụ', 'Mkpọọ'], 'pronunciati...","[{'igbo': 'Devid nwere mmaagha.', 'english': '...",['mmà aghā'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,war.,3,NaN
4,bawanye,Active verb,increase; grow in number,5f90c35e49f7e863e92b71df.webm,"[{'dialects': ['Izii'], 'pronunciation': '5f90...","[{'igbo': 'Ezinụụlọ anyị na-abawanye.', 'e...",NaN,ịbāwānyē,bàwanye,bàwànyèrè,NaN,nà-àbawanye,nà-àbawanye,gà-àbawanye,enter,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,àlà,Noun,land; ground; floor; the part of the earth's s...,5f90c35e49f7e863e92b711a.mp3,"[{'dialects': ['Nsa'], 'pronunciation': '5f90c...",[{'igbo': 'Ala ga-eme ka akụmakụ anyị too m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,land,5,NaN
96,yì akwà,Active verb,wear a cloth,63b12cb363110821906a80de.mp3,"[{'dialects': ['Mbaise', 'Ngwa'], 'pronunciati...","[{'igbo': 'Amadị yi akwa ọcha.', 'english': ...",NaN,iyì akwà,yìe akwà,yìrì akwà,NaN,nà-èyi akwà,nà-èyi akwà,gà-èyi akwà,Wear clothes.,5,NaN
97,-go,Extensional suffix,"perfect tense marker; has, have, had happened",61f9dad47d9b79750768332a.webm,"[{'dialects': ['Ọnịcha'], 'pronunciation': '61...","[{'igbo': 'Ọ bịago.', 'english': 'He has com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Go.,0,NaN
98,mùchi anya,Active verb,to close or shut one's eyes,63fe714045f2edf72f796dac.mp3,"[{'dialects': ['Nsụka'], 'pronunciation': '63f...",[{'igbo': 'Ọ muchiri anya ozigbo ọ hụrụ mm...,NaN,imùchi anya,mùchie anya,mùchìrì anya,NaN,nà-èmùchi anya,na-emùchi anya,gà-èmùchi anya,Close your eyes.,5,NaN


compare the definition with the translated text and provide a score from 0 to 5, where 0 completely different meaning  to 5 signifying exact match in terms of meaning

In [ ]:
definition = sample_data['definitions'][25]
translation = sample_data['translation'][25]

response = openai.chat.completions.create(
model="gpt-4o-mini",
messages=[
                {"role": "system", "content": "You are a helpful translator that translates Igbo to English."},
                {"role": "user", "content": f"compare the {definition} with the translated {translation}and provide a score from 0 to 5, where 0 completely different meaning to 5 signifying exact match in terms of meaning. Note that the definition may contain alternative meanings, so consider that when rating. Output only the score."}
          ],
            temperature=0.2
  )

score = response.choices[0].message.content.strip()


In [ ]:
score

'4'